### Figure 1: Predicted images

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import metpy.plots.ctables
from utils.data import load_tc_data
from utils.plot import make_cmap

# TODO: because i'm using imshow the lats need to be flipped - normally would be fine but because lats are flipped and imshow doesn't know this
# the TCs are plotted spinning the wrong way

sns.set_style("white")
sns.set_palette(sns.color_palette("Paired"))
sns.set_palette(sns.color_palette("Set2"))

In [2]:
# define cmap
precip_cmap,precip_norm = make_cmap()

In [3]:
def regrid(array):
        hr_array = np.zeros((100,100))
        for i in range(10):
                for j in range(10):
                        i1 = i*10
                        i2 = (i+1)*10
                        j1 = j*10
                        j2 = (j+1)*10
                        hr_array[i1:i2,j1:j2] = array[i,j]
        return hr_array

In [4]:
def plot_predictions(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_,plot='save'):
	if plot == 'save':
			fig, axes = plt.subplots(n, m, figsize=(5*m, 5*n), sharey=True)
	else:
			print('show')
			fig, axes = plt.subplots(n, m, figsize=(2*m, 2*n), sharey=True)

	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	for i in range(m):
		j = 0
		
		storm = storms[i]
		if storm in [3014,3487,3757,2651,3487,680,1438,3738]:
			print('THE STORM IS:',storm)
			print('storm max: ',real_x[storm].max())
			print('cnn max: ',pred_cnn_x[storm].max())
			print('vaegan max: ',pred_vaegan_x[storm].max())
			print('gan max: ',pred_gan_x[storm].max())
			print('inputs max: ', np.nanmax(inputs_x[storm]))
			print('\n')
		# norm = colors.Normalize(*range_)
		norm = precip_norm
		if i in [0,1,2]:
			axes[j,i].imshow(regrid(inputs[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j,i].invert_yaxis()
			if i == 1 or i ==0:
				# axes[j+1,i].imshow(np.flip(pred_cnn[storm],axis=0), interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
				axes[j+1,i].imshow(pred_cnn[storm-1], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			else:
				axes[j+1,i].imshow(pred_cnn[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+1,i].invert_yaxis()
			axes[j+2,i].imshow(pred_vaegan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].invert_yaxis()
			axes[j+3,i].imshow(pred_gan[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].invert_yaxis()
			im = axes[j+4,i].imshow(real[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+4,i].invert_yaxis()
		else:
			axes[j,i].imshow(regrid(inputs_x[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
			axes[j,i].invert_yaxis()
			axes[j+1,i].imshow(pred_cnn_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+1,i].invert_yaxis()
			axes[j+2,i].imshow(pred_vaegan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+2,i].invert_yaxis()
			axes[j+3,i].imshow(pred_gan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+3,i].invert_yaxis()
			im = axes[j+4,i].imshow(real_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
			axes[j+4,i].invert_yaxis()

		axes[j,i].set(xticklabels=[])
		axes[j,i].set(yticklabels=[])
		axes[j+1,i].set(xticklabels=[])
		axes[j+1,i].set(yticklabels=[])
		axes[j+2,i].set(xticklabels=[])
		axes[j+2,i].set(yticklabels=[])
		axes[j+3,i].set(xticklabels=[])
		axes[j+3,i].set(yticklabels=[])
		axes[j+4,i].set(xticklabels=[])
		axes[j+4,i].set(yticklabels=[])


		axes[j,0].set_ylabel('LR input',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+1,0].set_ylabel('U-NET',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+2,0].set_ylabel('VAEGAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+3,0].set_ylabel('WGAN',rotation=0,horizontalalignment='left',fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')
		axes[j+4,0].set_ylabel('HR obs',horizontalalignment='left',rotation=0,fontsize=48,labelpad=240,fontweight='bold',color='#56a1d7')

		axes[4,j].set_xlabel('Maria (2017; NA)',fontsize=24,labelpad=5)
		axes[4,j+1].set_xlabel('Haiyan (2013; WP)',fontsize=24,labelpad=5)
		axes[4,j+2].set_xlabel('Barbara (1995; EP)',fontsize=24,labelpad=5)
		axes[4,j+3].set_xlabel('Vayu (2019; NI)',fontsize=24,labelpad=5)
		axes[4,j+4].set_xlabel('Ursula (1998; SP)',fontsize=24,labelpad=5)
		axes[4,j+5].set_xlabel('Brendan (1985; WP)',fontsize=24,labelpad=5)

	fig.subplots_adjust(right=0.8)
	# cbar_ax = fig.add_axes([1, 0.15, 0.05, 0.7])
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	levels=[0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70, 100, 150]
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.01,ticks=levels)
	cb.ax.tick_params(labelsize=36)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('figure_1_new.png',bbox_inches='tight',dpi=600)
			plt.clf()
	else:
			plt.show()
	

In [5]:
# load data
real,inputs,pred_cnn,pred_vaegan,pred_gan,pred_vaegan_ensemble,pred_gan_ensemble,meta = load_tc_data(set='validation',results='test')
real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_valid',results='test')
real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_test',results='test')


meta = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')
meta_extreme = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_valid_meta.csv')
meta_extreme_test = pd.read_csv('/user/work/al18709/tc_data_mswep/extreme_test_meta.csv')
meta_test = pd.read_csv('/user/work/al18709/tc_data_mswep/test_meta.csv')
meta_train = pd.read_csv('/user/work/al18709/tc_data_mswep/train_meta.csv')
meta_valid = pd.read_csv('/user/work/al18709/tc_data_mswep/valid_meta.csv')

In [6]:
# take only top 100 extremes
inputs_x_max = np.max(inputs_x,axis=(1,2))
max_idx = list(np.argpartition(inputs_x_max, -100)[-100:])

# redefine extremes by only the most extreme samples
# inputs_hr_x = inputs_hr_x[max_idx]
print(np.mean(np.abs(np.max(pred_cnn_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
print(np.mean(np.abs(np.max(pred_vaegan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
print(np.mean(np.abs(np.max(pred_gan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))

# print(np.max(np.abs(np.max(pred_cnn_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_vaegan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_gan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))

# print(list(np.max(real_x[max_idx],axis=(1,2))))
# print(list(np.max(pred_cnn_x[max_idx],axis=(1,2))))

idx = list(np.argpartition(np.max(real_x[max_idx],axis=(1,2)), -10)[-10:])
print(np.max(real_x[max_idx][idx],axis=(1,2)))
print(np.max(pred_cnn_x[max_idx][idx],axis=(1,2)))
print(np.max(pred_vaegan_x[max_idx][idx],axis=(1,2)))
print(np.max(pred_gan_x[max_idx][idx],axis=(1,2)))
print('input max',np.max(inputs_x[max_idx][idx],axis=(1,2)))
print('real max',np.max(real_x[max_idx][idx],axis=(1,2)))
print(idx)

print(np.mean(np.abs(np.max(pred_cnn_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
print(np.mean(np.abs(np.max(pred_vaegan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
print(np.mean(np.abs(np.max(pred_gan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))

idx = list(np.argpartition(np.max(real_x[max_idx],axis=(1,2)), -10)[:90])
print(idx)
print(np.mean(np.abs(np.max(pred_cnn_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
print(np.mean(np.abs(np.max(pred_vaegan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
print(np.mean(np.abs(np.max(pred_gan_x[max_idx][idx],axis=(1,2))-np.max(real_x[max_idx][idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_vaegan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))
# print(np.max(np.abs(np.max(pred_gan_x[max_idx],axis=(1,2))-np.max(real_x[max_idx],axis=(1,2)))))

# print(np.max(inputs_x[max_idx],axis=(1,2)))
# print(np.max(pred_cnn_x[max_idx],axis=(1,2)))
# print(np.max(pred_vaegan_x[max_idx],axis=(1,2)))
# print(np.max(pred_gan_x[max_idx],axis=(1,2)))
# pred_vaegan_x = pred_vaegan_x[max_idx]
# pred_gan_x = pred_gan_x[max_idx]
# real_x = real_x[max_idx]
# pred_vaegan_ensemble_x = pred_vaegan_ensemble_x[max_idx]
# pred_gan_ensemble_x = pred_gan_ensemble_x[max_idx]

# print(np.sort(max_idx))

69.56123733520508
991.6434465026855
51.98715637207031
[380.5625 389.5    465.4375 519.1875 415.875  698.125  551.125  420.75
 599.0625 432.9375]
[212.63585 306.15155 122.6275  180.29596 268.4094  119.70642 160.23305
 144.94547 160.14088 226.53166]
[ 5052.73583984 82030.03125      148.0091095    450.50650024
  1756.95178223   147.12545776   408.32556152   352.35513306
   215.21656799  1244.85864258]
[310.87039185 515.62774658 189.84194946 297.20397949 399.30270386
 258.60092163 199.1386261  194.1378479  187.06108093 330.83059692]
input max [141.71022034 190.38748169  73.1897049  109.17042542 169.81555176
  71.50953674  96.30603027  93.66628265  75.9552002  119.88677979]
real max [380.5625 389.5    465.4375 519.1875 415.875  698.125  551.125  420.75
 599.0625 432.9375]
[70, 89, 42, 81, 79, 5, 96, 97, 4, 78]
297.0884765625
8999.78516845703
224.22021484375
[73, 49, 2, 3, 98, 95, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 92, 23, 24, 25, 91, 27, 28, 29, 30, 31, 32, 33, 34, 

In [7]:

plot = 'save'
n = 5
m = 6

# make colours
range_ = (0, 20)
storms = [4077, 4078, 4079, 4080, 4081, 4082]

# maria 8453 (8452 for unet) 8463
# haiyan 9820, 9828,  9845, 9840 9815, 9816, 9822 9809
# barb 1347 1375 1399
# georges 4082

storms = [8453,9845,1375,4128,5358,747]
storms = [8453,9845,1375,3014,3487,3757] 
storms = [8453,9845,1375,3014,3487,3738]
# storms_extremes = [4082,5349,763]
plot_predictions(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_)


THE STORM IS: 3014
storm max:  116.75
cnn max:  87.8419
vaegan max:  170.15652465820312
gan max:  127.39314270019531
inputs max:  76.67473602294922


THE STORM IS: 3487
storm max:  96.0625
cnn max:  86.41392
vaegan max:  161.053466796875
gan max:  110.26844787597656
inputs max:  68.79072570800781


THE STORM IS: 3738
storm max:  148.9375
cnn max:  108.175224
vaegan max:  181.9921417236328
gan max:  146.2998809814453
inputs max:  79.16040802001953




/tmp/ipykernel_42993/1396133910.py:82: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


<Figure size 3000x2500 with 0 Axes>

In [8]:
# print(meta.shape)
# print(real.shape) 

# print(np.sum(meta['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme['sid'] == '1992044S09181'))
# print(np.sum(meta_test['sid'] == '1992044S09181'))
# print(np.sum(meta_extreme_test['sid'] == '1992044S09181'))
# print(np.sum(meta_train['sid'] == '1992044S09181'))


# print(meta_extreme['sid'].values)
# for i,storm in enumerate(meta_extreme['sid'].values):
# 	pre_storm = meta_extreme['sid'].values[i-1]
# 	if storm != pre_storm:
# 		print(storm)

# Dorian in the test set
# Isaac (2012) '2012234N16315' training set NA
# Amphan 2020136N10088 ****** lost NI
# Harvey 2017228N14314 train NA
# Irma 2017242N16333 train NA
# Idai 2019063S18038 train SIO 
# Winston 2016041S14170 test SP
# Matthew 2016273N13300 extreme_test NA
# Jalawat 2012263N15141 extreme_test WP
# irene 2011233N15301 valid
# isaac 2012234N16315 train


# EP NA WP
# NIO SIO SP

# Maria 2017260N12310 validation NA
# Haiyan 2013306N07162 validation WP
# Barbara 1995189N11257 validation EP
print(np.where(meta_extreme['sid'] == '2020138N10086'))
print(np.where(meta_valid['sid'] == '2020138N10086'))
print(np.where(meta_train['sid'] == '2020138N10086'))
print(np.where(meta_test['sid'] == '2020138N10086'))
print(np.where(meta_extreme_test['sid'] == '2020138N10086'))
print(np.where(meta_extreme_valid['sid'] == '2020138N10086'))



# Georges 1998259N10335 extreme valid NA
# Daman 1992044S09181 extreme valid SP
# Haleh 2019060S10078 extreme valid SIO


# accumulated:
# Irene 2011233N15301 validation NA
# Haiyan 2013306N07162 validation WP
# Pauline 1997279N12263 extreme test EP
# Yasa 2020348S14173 SP (fiji)
# Idai 2019063S18038 train SI or Haleh 2019060S10078 extreme valid SI
# Amphan 2020136N10088 NI 2020138N10086

# check
# Harvey 2017228N14314 train NA
# Isaac 2012234N16315 train NA
# Maria 2017260N12310 validation NA
# Irma 2017242N16333 train NA
# Dorian 2019236N10314 test NA

(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)
(array([], dtype=int64),)


In [9]:
tc_list = [680,1438,2651,3014,3487,3738,3757]

for i in tc_list:
	print(meta_extreme_test['sid'][i])

1985298N12249
1990171N11148
1997013S08101
1998259N17118
2003240N20139
2004156N16117
2004164N06139


In [10]:
# 2019160N11073 - VAYU 2019 NI 3014
# 2012346S14180 - Evan 2013 SP 2857
# 2005042S12190 - Nancy 2005 SP 2651

# 3014 VAYU 3014
# 3487 Ursula 1998 SP
# 3738 Typhoon Brenden 1985 WP

# 3757 Cyclone Ofa 1990 SP

# 680


In [11]:
print(np.max(inputs))
print(np.max(real))

65.83993530273438
286.1875


In [12]:
print(np.max(inputs_x))
print(np.max(real_x))

190.38748168945312
698.125


In [13]:
def plot_haleh(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_,plot='save'):

	fig, axes = plt.subplots(1, 3, figsize=(30, 10), sharey=True)
	pal = sns.cubehelix_palette(10, rot=-.25, light=.7,as_cmap=True)

	storm=storms
	norm = precip_norm
	axes[0].imshow(regrid(inputs_x[storm]), interpolation='nearest', norm=norm,extent=None,cmap=precip_cmap)
	axes[0].invert_yaxis()
	axes[1].imshow(pred_vaegan_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
	axes[1].invert_yaxis()
	im = axes[2].imshow(real_x[storm], interpolation='nearest',norm=norm, extent=None,cmap=precip_cmap)
	axes[2].invert_yaxis()

	axes[0].set(xticklabels=[])
	axes[0].set(yticklabels=[])
	axes[1].set(xticklabels=[])
	axes[1].set(yticklabels=[])
	axes[2].set(xticklabels=[])
	axes[2].set(yticklabels=[])


	# axes[2].set_xlabel('Haleh (2019)',fontsize=24,labelpad=5)

	fig.subplots_adjust(right=0.8)
	cbar_ax = fig.add_axes([1, 0.031, 0.03, 0.936])
	levels=[0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 70, 100, 150]
	cb = fig.colorbar(im, cax=cbar_ax,shrink=0.005,ticks=levels)
	cb.ax.tick_params(labelsize=36)
	fig.tight_layout()

	if plot == 'save':
			plt.savefig('tropical_cyclone_haleh.png',bbox_inches='tight',dpi=600)
			plt.clf()
	else:
			plt.show()
	

In [14]:
# real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_valid',results='test')
real_x,inputs_x,pred_cnn_x,pred_vaegan_x,pred_gan_x,pred_vaegan_ensemble_x,pred_gan_ensemble_x,meta_extreme_valid = load_tc_data(set='extreme_test',results='test')


In [15]:
storms = [8453,9845,1375,3014,3487,3757]
storms = [8453,9845,1375,4128,5358,747]
# print(max_idx)
storms = 1624
pred_vaegan_x_max = np.max(pred_vaegan_x,axis=(1,2))
max_idx = list(np.argpartition(pred_vaegan_x_max, -10)[-10:])
print(max_idx)
range_ = (0, 20)
plot_haleh(inputs,inputs_x,pred_cnn,pred_cnn_x,pred_vaegan,pred_vaegan_x,pred_gan,pred_gan_x,real,real_x,storms,range_)


[4812, 3913, 4434, 4650, 1628, 5063, 4866, 4805, 4798, 1624]


/tmp/ipykernel_42993/2158126563.py:30: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


<Figure size 3000x1000 with 0 Axes>